<span style="color:#8735fb; font-size:24pt"> **Multi-GPU example on Azure using dask-cuda** </span>

[Dask Cuda](https://dask-cuda.readthedocs.io/en/latest/) is a library which extends Dask's distributed single machine [LocalCluster](https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster) for use in workloads that can leverage multiple GPUs.  
In this notebook, we explore how we can leverage multiple GPUs in a single node using XGboost. 
We will further explore how we can use ForestInference library of cuml to do inference. 

For the purposes of this demo, we will use the a part of the NYC Taxi Dataset (only the files of 2018 calendar year will be used here). The goal is to predict the fare amount for a given trip given the times and coordinates of the taxi trip.

### <span style="color:#8735fb; font-size:22pt"> **Step 0: Import Stuff** </span>

#### TODO: Installs a bunch of stuff as dependencies including pyspark ! screwing up the environment. Need to look for possible alternative sources for the data. Otherwise need to run this in a docker container. 


In [ ]:
# # If azureml is not present install azureml-core. 
# # Further opendatasets is in preview mode hence it is not available in the main sdk and needs to be installed separately.
# # Installing azureml-opendatasets with for some weird reason install Pandas 1.0.0 which is not what we need. So we install back 
# # pandas 1.2.4 after we are done.
# ! pip install azureml-core
# ! pip install azureml-opendatasets
# ! pip install azureml-telemetry
# ! pip install pandas==1.2.4 # reverting pandas to 1.2.4

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait
import dask_cudf
from dask_ml.model_selection import train_test_split
from cuml.dask.common import utils as dask_utils
from cuml.metrics import mean_squared_error
from cuml import ForestInference
import cudf
import xgboost as xgb
from azureml.opendatasets import NycTlcYellow
from datetime import datetime
from dateutil import parser
import numpy as np
from timeit import default_timer as timer

<span style="color:#8735fb; font-size:22pt"> **Step 1: Set up the Local CUDA Cluster** </span>

In [ ]:
# Set the number of partitions to 8 for Dask. Depending on how many GPUs you have, you can set the number of partitions. 
# right now someone is doing something on CUDA 0
CUDA_VISIBLE_DEVICES = [3,4,5,6,7]
npartitions = 5 #8
clust = LocalCUDACluster(host="", 
                         CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                         n_workers=npartitions,
                         scheduler_port = 8786)
client = Client(clust)
clust

In [ ]:
client

In [ ]:
def pretty_print(scheduler_dict):
    print(f"All workers for scheduler id: {scheduler_dict['id']}, address: {scheduler_dict['address']}")
    for worker in scheduler_dict['workers']:
        print(f"Worker: {worker} , gpu_machines: {scheduler_dict['workers'][worker]['gpu']}")

pretty_print(client.scheduler_info()) # will show information on the len(CUDA_VISIBLE_DEVICES) partitions

<span style="color:#8735fb; font-size:22pt"> **Step 2: Data Setup, Cleanup and Enhancement** </span>

### <span style="color:#8735fb; font-size:18pt"> Step 2.a: Download the Data from Azureml Opendatasets </span>

In [ ]:
tic = timer()    
start_date = parser.parse('2018-05-01') # lets start at 1st May 2018
end_date = parser.parse('2018-06-06') # Lets stop at 6th June 2018
nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
nyc_tlc_df = nyc_tlc.to_pandas_dataframe()
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

In [ ]:
print(nyc_tlc_df.shape) # Apprx. 10 Million rows
print(type(nyc_tlc_df))
print(nyc_tlc_df.head()) # does not work without pandas 1.0.0 which gets installed with azure-opendatasets

In [ ]:
df = dask_cudf.from_cudf(cudf.from_pandas(nyc_tlc_df), npartitions=npartitions)

Let's look at the data locally to see what we're dealing with. We will make use of the data from 2018 for the purposes of the demo. We see that there are columns for pickup and dropoff times, distance, along with latitude, longitude, etc. These are the information we'll use to estimate the trip fare amount.

### <span style="color:#8735fb; font-size:18pt"> Step 2.b: Data Cleanup Scripts </span>

The data needs to be cleaned up before it can be used in a meaningful way. We first perform a renaming of some columns to a cleaner name (for instance, some of the years have `tpep_ropoff_datetime` instead of `dropfoff_datetime`). We also define the datatypes each of the columns need to be read as.

In [ ]:
#create a list of columns & dtypes the df must have
must_haves = {
 'tpepPickupDateTime': 'datetime64[ms]',
 'tpepDropoffDateTime': 'datetime64[ms]',
 'passengerCount': 'int32',
 'tripDistance': 'float32',
 'startLon': 'float32',
 'startLat': 'float32',
 'rateCodeId': 'int32',
 'endLon': 'float32',
 'endLat': 'float32',
 'fareAmount': 'float32'
}

def clean(df_part, must_haves):
    """
    This function performs the various clean up tasks for the data
    and returns the cleaned dataframe.
    """
    # iterate through columns in this df partition
    for col in df_part.columns:
        # drop anything not in our expected list
        if col not in must_haves:
            df_part = df_part.drop(col, axis=1)
            continue

        # fixes datetime error found by Ty Mckercher and fixed by Paul Mahler
        if df_part[col].dtype == 'object' and col in ['tpepPickupDateTime', 'tpepDropoffDateTime']:
            df_part[col] = df_part[col].astype('datetime64[ms]')
            continue

        # if column was read as a string, recast as float
        if df_part[col].dtype == 'object':
            df_part[col] = df_part[col].str.fillna('-1')
            df_part[col] = df_part[col].astype('float32')
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if 'int' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('int32')
            if 'float' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('float32')
            df_part[col] = df_part[col].fillna(-1)
    return df_part

### <span style="color:#8735fb; font-size:18pt"> Step 2.c: Data Enhancements, Add interesting Features </span>

We'll add new features by making use of "uder defined functions" on the dataframe. We'll make use of [apply_rows](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.apply_rows), which is similar to Pandas' apply funciton. `apply_rows` operation is [JIT compiled by numba](https://numba.pydata.org/numba-doc/dev/cuda/kernels.html) into GPU kernels. 

The kernels we define are - 
1. Haversine distance: This is used for calculating the total trip distance.

2. Day of the week: This can be useful information for determining the fare cost.

`add_features` function combined the two to produce a new dataframe that has the added features.

In [ ]:
import math
from math import cos, sin, asin, sqrt, pi

def haversine_distance_kernel(startLat, startLon, endLat, endLon, h_distance, radius):
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(startLat, startLon, endLat, endLon,)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_2 = pi/180 * x_2
        y_2 = pi/180 * y_2
        
        dlon = y_2 - y_1
        dlat = x_2 - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_2) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        # radius = 6371 # Radius of earth in kilometers # currently passed as input arguments
        
        h_distance[i] = c * radius

def day_of_the_week_kernel(day, month, year, day_of_week):
    for i, (d_1, m_1, y_1) in enumerate(zip(day, month, year)):
        if month[i] <3:
            shift = month[i]
        else:
            shift = 0
        Y = year[i] - (month[i] < 3)
        y = Y - 2000
        c = 20
        d = day[i]
        m = month[i] + shift + 1
        day_of_week[i] = (d + math.floor(m*2.6) + y + (y//4) + (c//4) -2*c)%7
        
def add_features(df):
    df['hour'] = df['tpepPickupDateTime'].dt.hour
    df['year'] = df['tpepPickupDateTime'].dt.year
    df['month'] = df['tpepPickupDateTime'].dt.month
    df['day'] = df['tpepPickupDateTime'].dt.day
    df['diff'] = (df['tpepDropoffDateTime'] - df['tpepPickupDateTime']).dt.seconds #convert difference between pickup and dropoff into seconds
    
    df['pickup_latitude_r'] = df['startLat']//.01*.01
    df['pickup_longitude_r'] = df['startLon']//.01*.01
    df['dropoff_latitude_r'] = df['endLat']//.01*.01
    df['dropoff_longitude_r'] = df['endLon']//.01*.01
    
    df = df.drop('tpepDropoffDateTime', axis=1)
    df = df.drop('tpepPickupDateTime', axis =1)
    
    
    df = df.apply_rows(haversine_distance_kernel,
                   incols=['startLat', 'startLon', 'endLat', 'endLon'],
                   outcols=dict(h_distance=np.float32),
                   kwargs=dict(radius=6371))
    
    
    df = df.apply_rows(day_of_the_week_kernel,
                      incols=['day', 'month', 'year'],
                      outcols=dict(day_of_week=np.float32),
                      kwargs=dict())
    
    
    df['is_weekend'] = (df['day_of_week']<2)
    return df


`df` is a dask dataframe. 

In [ ]:
df

In [ ]:
# Query the dataframe to clean up the outliers 
df = clean(df, must_haves)
# Add new features to all the partitions of the dataframe
taxi_df = df.map_partitions(add_features)
# Drop NaN values and convert to float32
taxi_df = taxi_df.dropna()
taxi_df = taxi_df.astype("float32")

print(type(taxi_df)) # still delayed

### <span style="color:#8735fb; font-size:18pt"> Step 2.d: Split Data </span>

In [ ]:
X, y = taxi_df.drop(["fareAmount"], axis=1), taxi_df["fareAmount"].astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=12)

### <span style="color:#8735fb; font-size:18pt"> Step 2.e: Persist Data across workers </span>

In [ ]:
workers = client.has_what().keys()
X_train, X_test, y_train, y_test = dask_utils.persist_across_workers(client,
                                                       [X_train, X_test, y_train, y_test],
                                                       workers=workers)
wait([X_train, X_test, y_train, y_test])

In [ ]:
# X_train.visualize() # has len(CUDA_VISIBLE_DEVICES) partitions

<span style="color:#8735fb; font-size:22pt"> **Step 3: Train a XGBoost Model** </span>

We are now ready to fit a XGBoost model on the data to predict the fare for the trip.

Always good to check what is the condition of the GPUs you have.

In [ ]:
!nvidia-smi

### <span style="color:#8735fb; font-size:18pt"> Step 3.a: Set training Parameters </span>

We will use the eval metrix RMSE for this problem. Note that for better performance we should perform HPO ideally to get the best parameters. 

Refer to the notebooks in the repository for how to perform automated HPO [using RayTune](https://github.com/rapidsai/cloud-ml-examples/blob/main/ray/notebooks/Ray_RAPIDS_HPO.ipynb) and [using Optuna](https://github.com/rapidsai/cloud-ml-examples/blob/main/optuna/notebooks/optuna_rapids.ipynb).

In [ ]:
params = {
    'learning_rate': 0.15,
    'max_depth': 11,
    'objective': 'reg:squarederror',
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 1,
    'gamma': 1,
    'silent': True,
    'verbose_eval': True,
    'booster' : 'gbtree', # 'gblinear' not implemented in dask
    'eval_metric': 'rmse',
    'tree_method':'gpu_hist',
    'num_boost_rounds': 100
}


### <span style="color:#8735fb; font-size:18pt"> Step 3.b: Train XGBoost Model </span>

In [ ]:
tic = timer()
data_train = xgb.dask.DaskDMatrix(client, X_train, y_train)
xgboost_output = xgb.dask.train(client, params,data_train, 
                                    num_boost_round=params['num_boost_rounds'])
xgb_cuda_model = xgboost_output['booster']
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

### <span style="color:#8735fb; font-size:18pt"> Step 3.c: Save the Model to disk </span>

In [ ]:
model_filename = 'trained-model_nyctaxi.xgb'
xgb_cuda_model.save_model(model_filename)

### <span style="color:#8735fb; font-size:22pt"> **Step 4: Predict & Score using vanilla XGBoost Predict** </span>

In [ ]:
tic = timer()
d_test = xgb.dask.DaskDMatrix(client, X_test)
y_pred = xgb.dask.predict(client, xgb_cuda_model, d_test)
_y_pred, _y_test = y_pred.compute(), y_test.compute()
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

In [ ]:
tic = timer()
print("Calculating MSE")
score = mean_squared_error(_y_pred, _y_test)
print("Workflow Complete - RMSE: ", np.sqrt(score))
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

### <span style="color:#8735fb; font-size:22pt"> **Step 5: Predict & Score using FIL or Forest Inference Library** </span>

[ForestInference](https://docs.rapids.ai/api/cuml/stable/api.html?highlight=forestinference#cuml.ForestInference) provides GPU accelerated inference capabilities for tree models. 
It accepts a **trained** tree model in a treelite format (currently LightGBM, XGBoost and SKLearn GBDT and random forest models
are supported). 
However, you cannot use it to train anything. 

In [ ]:
from cuml import ForestInference

Here if the `X_test` is small enough we can call `compute()` on it. But in general, if the `X_test` is quite large, it is better to persist it on the different workers and then call the predict on the individual workers separately. We we show both. 

### <span style="color:#8735fb; font-size:18pt"> Step 5.a: Predict using `compute` on a single worker in case the test dataset is small. </span>

In [ ]:
tic = timer()
X_test_computed = X_test.compute()
loaded_model = ForestInference.load(model_filename, model_type='xgboost')
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

In [ ]:
tic = timer()
fil_pred = loaded_model.predict(X_test_computed)
print("Final - RMSE: ", np.sqrt(score))
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

In [ ]:
tic=timer()
score = mean_squared_error(fil_pred, _y_test)
print("Final - RMSE: ", np.sqrt(score))
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

Compare the RMSE results with that obtained without using FIL. You would see they are approximately same.

### <span style="color:#8735fb; font-size:18pt"> Step 5.b: Predict using `persist` on multiple workers in case the test dataset is huge. </span>

In [ ]:
from dask.distributed import get_worker

In [ ]:
workers = client.has_what().keys()
print(workers)
n_workers = len(workers)
n_partitions = n_workers

In [ ]:
def worker_model_init(model_file):   
    # this function will run in each worker and initialize the worker 
    worker = get_worker()
    worker.data["fil_model"] = ForestInference.load(filename=model_file,model_type='xgboost')
    
def predict(input_df):
    # this function will run in each worker and predict 
    worker = get_worker()
    return worker.data["fil_model"].predict(input_df)

Persist the `X_test` in the workers if not already persisted. 

In [ ]:
# X_test = X_test.persist()
# wait(X_test)

In [ ]:
tic= timer()
client.run(worker_model_init, model_filename)
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

In [ ]:
tic = timer()
predictions = X_test.map_partitions(predict, meta="float") # this is like MPI reduce
y_pred = predictions.compute()
toc = timer()
print(f"Wall clock time taken for this cell : {toc-tic} s")

In [ ]:
tic = timer()
score = mean_squared_error(y_pred, _y_test)
toc = timer()
print("Final - RMSE: ", np.sqrt(score))

In [ ]:
len(y_pred)

### <span style="color:#8735fb; font-size:22pt"> **Step 6: Clean up** </span>

In [ ]:
client.close()
clust.close()